In [1]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from tqdm import tqdm
from pprint import pprint

In [ ]:
import os 

os.environ["HUGGINGFACEHUB_API_TOKEN"]=''

In [3]:
class LLMState(TypedDict):
    question : str
    answer : str

In [4]:
llm = HuggingFaceEndpoint(
    repo_id="moonshotai/Kimi-K2-Instruct-0905",
    task="text-generation",
    temperature=0.6,  # Recommended by Moonshot AI
    max_new_tokens=4096,
)

chat_model = ChatHuggingFace(
    llm = llm
)


def llm_node(state: LLMState) -> LLMState:
    que = state['question']

    messages = [
        {"role": "system", "content": "You are a great teacher just like Richard Feymann, you have to teach python concepts with indepth knowledge and with multiple examples ranging from basic ones to advanced ones"},
        {"role": "user", "content": que}
    ]

    ans = chat_model.invoke(messages).content

    state['answer'] = ans

    return state

/home/rajgajjar04/Learnings/Langchain_Langgraph/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
graph = StateGraph(LLMState)

graph.add_node("llm_node", llm_node)

graph.add_edge(START, "llm_node")
graph.add_edge("llm_node", END)

workflow = graph.compile()

In [6]:
initial_state = {"question": "Teach me about python decorators in depth"}

final_state = workflow.invoke(initial_state)

print(final_state['answer'])

Okay, let’s pretend we’re sitting in a lab with a white-board.  
Richard Feynman once said “*What I cannot create, I do not understand*”.  
So we will **build** decorators from the ground up—no syntactic sugar at first—until the `@` symbol becomes a trivial shorthand for something we already understand.

------------------------------------------------
0.  Mental model: a decorator is **a function that gets a function and returns a function**.  
    Nothing else.  Everything else is just details.

------------------------------------------------
1.  The smallest possible decorator
------------------------------------------------
```python
def loud(func):
    def wrapper(*args, **kwargs):
        print("calling", func.__name__)
        result = func(*args, **kwargs)
        print("finished", func.__name__)
        return result
    return wrapper

def greet(name):
    print("hello", name)

greet = loud(greet)   # manual decoration
greet("Alice")
```
Output
```
calling greet
hello Alice
